<a href="https://colab.research.google.com/github/IagoConrado/colab-notebooks/blob/master/Agrupamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Previsão de insuficiência cardíaca**


*   Link do dataset: [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data).
*   Os dados do dataset descrevem características dos pacientes e se eles faleceram durante o acompanhamento.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Imports**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)